In [ ]:
from langchain_google_community import (
    VertexAISearchRetriever,
)

PROJECT_ID = "inbound-mote-433711-a6"
LOCATION_ID = "global"
SEARCH_ENGINE_ID = "alphabet-investor-pdfs_1725497923032"
DATA_STORE_ID = "alphabet-investor-pdfs_1725497426427"

In [ ]:
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION_ID,
    data_store_id=DATA_STORE_ID,
    max_documents=3,
)

query = "What are Alphabet's Other Bets?"

result = retriever.invoke(query)
for doc in result:
    print(doc)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_vertexai import ChatVertexAI

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

    Context: {context}
    
    Question: {question}"""
)

llm = ChatVertexAI(model="gemini-1.5-flash-001", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke(query))